## pytorch를 통한 CNN 구현
- task : 이미지 다중분류 (레이블 5개)
- 참고 (model.train(), model.eval(), torch.no_grad()란?) : https://tigris-data-science.tistory.com/entry/PyTorch-modeltrain-vs-modeleval-vs-torchnograd

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from tqdm import tqdm
import torchsummary

In [2]:
import tensorflow as tf
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True,
                                   )
data_dir = pathlib.Path(data_dir)

228827136/228813984 [==============================] - 1s 0us/step


### DataLoader
- validation set 경우 shuffle False
- test set과 마찬가지로 torch.no_grad()

In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import transforms as T
t = T.Compose([T.Resize((224,224)),T.ToTensor(),T.Normalize((0.5,0.5,0.5), (0.25,0.25,0.25))])
dataset = ImageFolder(root = data_dir,transform = t)
print(dataset.__len__()) # Dasiy, dandelion(민들레), roses, sunflowers, tulibs이란 라벨을 가진 총 3670장의 이미지

3670


In [4]:
train_size = int(dataset.__len__() * 0.8)
valid_size = dataset.__len__() - train_size

training_data, valid_data = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False) # valid는 섞지않는다!

### 모델 정의
- 고려사항
    - 모델의 정도: 깊게 쌓을 것인가(Layer를 늘릴지), 혹은 넓게 쌓을 것인가(Filter를 늘릴지), kernal_size를 어떻게 조절할 것인지?
    - Dropout: Overfitting을 어떻게 방지할 것인가?
    - Batch Normalization: 배치 단위로 정규화를 넣어줄 것인지
    - Activation function: 다른걸 써볼 수 있을지

In [5]:
class myCNN(nn.Module):
    def __init__(self):
        super(myCNN, self).__init__()
        self.layer1 = nn.Sequential(
            # Sequential : 하나의 층당 다음과정을 순서대로 수행 (conv => 활성화함수 => pooling)
            # RGB 3채널의 이미지를 입력으로 받아서, 이미지 하나당 32개의 필터뱅크 적용 (커널사이즈 3짜리)
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(2, 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2)
        )
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 5) # 분류해야 할 이미지가 5종류
    
    def forward(self, x):
        # forward : 연산순서 정의
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 64*56*56)
        x = F.relu(self.fc1(x)) # 여긴 F.relu인것을 주목 nn.ReLU가 아니라
        return self.fc2(x)

### 손실함수, 옵티마이저 설정
- Loss: 어떤 loss를 사용할 것인가?
- Optim: SGD 외에 RMSprop, Adam 등으로 바꾸면 학습에 어떤 영향을 미치는가?
- Learning rate: 어느정도의 속도로 학습할 것인가?

In [6]:
gpu = torch.device("cuda")
model = myCNN().to('cpu')
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

import torchsummary
torchsummary.summary(model, (3, 224, 224)) # 224*224 RGB 이미지

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
         MaxPool2d-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 64, 112, 112]          18,496
              ReLU-5         [-1, 64, 112, 112]               0
         MaxPool2d-6           [-1, 64, 56, 56]               0
            Linear-7                  [-1, 128]      25,690,240
            Linear-8                    [-1, 5]             645
Total params: 25,710,277
Trainable params: 25,710,277
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 41.34
Params size (MB): 98.08
Estimated Total Size (MB): 140.00
----------------------------------------------------------------


### 학습 돌리기

In [7]:
epochs = 5
torch.cuda.empty_cache()
train_losses = []
val_losses = []
train_acc = []
val_acc = []

for epoch_num in range(epochs):
    
    # 1. train
    model.train() # ★ layer들을 Training mode로 바꿔준다
    running_loss = 0
    running_accuracy = 0
    
    for _, data in enumerate(tqdm(train_loader)):
        # 준비물
        inputs, labels = data # 이미지, 라벨
        inputs = inputs.to(gpu).float()
        labels = inputs.to(gpu).long()
        optimizer.zero_grad() # ★주의
        
        # 순전파
        outputs = model(inputs) # 이미지 모델에 넣기 => 값 : 다중확률
        print(f'outputs : {outputs}')
        _, preds = torch.max(outputs, 1) # 가장놓은걸 pred로 취급
        loss = loss_fn(outputs, labels)
        
        # 역전파
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_accuracy += torch.sum(preds == labels.data).detach().cpu().numpy()/inputs.size()
    
    # 2. validation
    model.eval() # ★ layer들을 eval mode로 바꿔준다
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad(): # ★ 역전파를 안하기때문에 gradient 를 사용하지않는다
        for _, data in enumerate(tqdm(valid_loader)):
            # 준비물                
            inputs, labels = data
            inputs = inputs.to(gpu).float()
            labels = labels.to(gpu).long()

            # 순전파
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            # 역전파과정은 없음. 지표만 더해주기.
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            val_accuracy += torch.sum(preds == labels.data).detach().cpu().numpy()/inputs.size(0)
            
    # 3. 각 배치별 평균성능 계산하기
    train_losses.append(running_loss / len(train_loader))
    val_losses.append(val_loss / len(valid_loader))
    train_acc.append(running_accuracy / len(train_loader))
    val_acc.append(val_accuracy / len(valid_loader))
    
    print("Epoch:{}/{}..".format(epoch_num + 1, epochs),
            "Train Loss: {:.3f}..".format(running_loss / len(train_loader)),
            "Val Loss: {:.3f}..".format(val_loss / len(valid_loader)),
            "Train Acc:{:.3f}..".format(running_accuracy / len(train_loader)),
            "Val Acc:{:.3f}..".format(val_accuracy / len(valid_loader)))

history = {'train_loss': train_losses, 'val_loss': val_losses,
            'train_acc': train_acc, 'val_acc': val_acc}    

100%|██████████| 23/23 [00:03<00:00,  6.48it/s]


Epoch:1/5.. Train Loss: 1.266.. Val Loss: 1.146.. Train Acc:0.488.. Val Acc:0.548..


100%|██████████| 23/23 [00:03<00:00,  5.94it/s]


Epoch:2/5.. Train Loss: 0.978.. Val Loss: 1.032.. Train Acc:0.612.. Val Acc:0.564..


100%|██████████| 23/23 [00:03<00:00,  6.54it/s]


Epoch:3/5.. Train Loss: 0.809.. Val Loss: 1.147.. Train Acc:0.693.. Val Acc:0.553..


100%|██████████| 23/23 [00:03<00:00,  6.49it/s]


Epoch:4/5.. Train Loss: 0.663.. Val Loss: 0.983.. Train Acc:0.767.. Val Acc:0.636..


100%|██████████| 23/23 [00:03<00:00,  6.42it/s]

Epoch:5/5.. Train Loss: 0.534.. Val Loss: 1.023.. Train Acc:0.826.. Val Acc:0.606..


### 모델 평가

In [8]:
model.eval() # ★ eval 모드
y_pred = []
y_true = []

with torch.no_grad() : # ★ gradient 사용하지 않는다
    for _, data in enumerate(tqdm(valid_loader)):
        inputs, labels = data                    
        inputs = inputs.to(gpu).float()            
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)                        
        y_pred += list(preds.detach().cpu().numpy())
        y_true += list(labels.detach().numpy())

100%|██████████| 23/23 [00:03<00:00,  6.47it/s]


In [9]:
# 레이블별 confusion matrix를 통한 예측 리포트
from sklearn.metrics import classification_report, confusion_matrix
class_names = ['daisy','dandelion','roses','sunflowers','tulibs']
print('\n Classification report \n\n',
  classification_report(
      y_true,
      y_pred,
       target_names=class_names
      )
  )


 Classification report 

               precision    recall  f1-score   support

       daisy       0.68      0.42      0.52       118
   dandelion       0.52      0.86      0.65       177
       roses       0.53      0.55      0.54       141
  sunflowers       0.80      0.65      0.72       143
      tulibs       0.68      0.46      0.55       155

    accuracy                           0.61       734
   macro avg       0.64      0.59      0.60       734
weighted avg       0.64      0.61      0.60       734

